# Regras de associação.

Cálculos de itemsets frequentes com o algoritmo Apriori utilizando o pacote ```mlxt```.


Este notebook foi desenvolvido para o ambiente GOOGLE COLAB ([colab.research.google.com](https://colab.research.google.com)).

Prof. Hugo de Paula



In [2]:
! pip install mlxtend
! pip install xlrd

## Regras de associação geradas a partir de itemsets frequentes

Fonte: https://rasbt.github.io/mlxtend/user_guide/frequent_patterns/association_rules/

No exemplo a seguir, foi criado um ```dataset```  transacional formado por uma "lista de listas", onde cada linha corresponde a um cesto de compras de um supermercado hipotético.

Nesta base, são considerados ```itemsets``` frequentes aqueles que possuírem suporte superior a 0.6.

In [3]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

# Dataset transacional com cestos de compras

dataset = [['Leite', 'Cebola', 'Batata', 'Feijão', 'Ovos', 'Iogurte'],
           ['Arroz', 'Cebola', 'Batata', 'Feijão', 'Ovos', 'Iogurte'],
           ['Leite', 'Maçã', 'Feijão', 'Ovos'],
           ['Leite', 'Milho', 'Feijão', 'Iogurte'],
           ['Milho', 'Cebola', 'Feijão', 'Sorvete', 'Ovos']]

te = TransactionEncoder()
te_ary = te.fit(dataset).transform(dataset)
df = pd.DataFrame(te_ary, columns=te.columns_)
frequent_itemsets = apriori(df, min_support=0.6, use_colnames=True)

print(frequent_itemsets)

    support                itemsets
0       0.6                (Cebola)
1       1.0                (Feijão)
2       0.6               (Iogurte)
3       0.6                 (Leite)
4       0.8                  (Ovos)
5       0.6        (Feijão, Cebola)
6       0.6          (Ovos, Cebola)
7       0.6       (Feijão, Iogurte)
8       0.6         (Feijão, Leite)
9       0.8          (Feijão, Ovos)
10      0.6  (Feijão, Ovos, Cebola)


###Regras de associação

Gera regras de associação com confiança mínima de 0.7.

In [0]:
association_rules(frequent_itemsets, metric="confidence", min_threshold=0.7)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Cebola),(Feijão),0.6,1.0,0.6,1.00,1.00,0.00,inf
1,(Cebola),(Ovos),0.6,0.8,0.6,1.00,1.25,0.12,inf
2,(Ovos),(Cebola),0.8,0.6,0.6,0.75,1.25,0.12,1.6
3,(Iogurte),(Feijão),0.6,1.0,0.6,1.00,1.00,0.00,inf
4,(Leite),(Feijão),0.6,1.0,0.6,1.00,1.00,0.00,inf
5,(Feijão),(Ovos),1.0,0.8,0.8,0.80,1.00,0.00,1.0
6,(Ovos),(Feijão),0.8,1.0,0.8,1.00,1.00,0.00,inf
7,"(Feijão, Cebola)",(Ovos),0.6,0.8,0.6,1.00,1.25,0.12,inf
8,"(Feijão, Ovos)",(Cebola),0.8,0.6,0.6,0.75,1.25,0.12,1.6
9,"(Cebola, Ovos)",(Feijão),0.6,1.0,0.6,1.00,1.00,0.00,inf


Gera regras de associação com lift mínimo de 1.2. 

É importante lembrar que valores de lift inferiores a 1 significam que a regra não possui causalidade relevante e não aumentam o nosso poder de previsão.

In [0]:
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1.2)
print(rules)

        antecedents       consequents  ...  leverage  conviction
0          (Cebola)            (Ovos)  ...      0.12         inf
1            (Ovos)          (Cebola)  ...      0.12         1.6
2  (Feijão, Cebola)            (Ovos)  ...      0.12         inf
3    (Feijão, Ovos)          (Cebola)  ...      0.12         1.6
4          (Cebola)    (Feijão, Ovos)  ...      0.12         inf
5            (Ovos)  (Feijão, Cebola)  ...      0.12         1.6

[6 rows x 9 columns]


In [0]:
rules["antecedent_len"] = rules["antecedents"].apply(lambda x: len(x))
print(rules)

        antecedents       consequents  ...  conviction  antecedent_len
0          (Cebola)            (Ovos)  ...         inf               1
1            (Ovos)          (Cebola)  ...         1.6               1
2  (Feijão, Cebola)            (Ovos)  ...         inf               2
3    (Feijão, Ovos)          (Cebola)  ...         1.6               2
4          (Cebola)    (Feijão, Ovos)  ...         inf               1
5            (Ovos)  (Feijão, Cebola)  ...         1.6               1

[6 rows x 10 columns]


Exibe apenas as regras com antecedentes de comprimento maior ou igual a 2 e com confiança superior a 0.75 e lift superior a 1.2.

In [0]:
rules[ (rules['antecedent_len'] >= 2) &
       (rules['confidence'] > 0.75) &
       (rules['lift'] > 1.2) ]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,antecedent_len
2,"(Feijão, Cebola)",(Ovos),0.6,0.8,0.6,1.0,1.25,0.12,inf,2


Exibe apenas as regras cujos antecedentes são Feijão e Ovos.

In [0]:
rules[rules['antecedents'] == {'Ovos', 'Feijão'}]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,antecedent_len
3,"(Feijão, Ovos)",(Cebola),0.8,0.6,0.6,0.75,1.25,0.12,1.6,2


## Análise de cesta de compras em Python

Fonte:  Chris Moffitt (2017), Introduction to Market Basket Analysis in Python, http://pbpython.com/market-basket-analysis.html


Neste exemplo é utilizada a base de dados **Online Retail** da UCI, disponível em [archive.ics.uci.edu/ml/machine-learning-databases/00352/Online Retail.xlsx](http://archive.ics.uci.edu/ml/machine-learning-databases/00352/Online%20Retail.xlsx)

In [0]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

In [0]:
df = pd.read_excel('http://archive.ics.uci.edu/ml/machine-learning-databases/00352/Online%20Retail.xlsx')
print(df.head())

In [0]:
print(df.describe())

            Quantity      UnitPrice     CustomerID
count  532621.000000  532621.000000  397924.000000
mean       10.239972       3.847621   15294.315171
std       159.593551      41.758023    1713.169877
min     -9600.000000  -11062.060000   12346.000000
25%         1.000000       1.250000   13969.000000
50%         3.000000       2.080000   15159.000000
75%        10.000000       4.130000   16795.000000
max     80995.000000   13541.330000   18287.000000


### Preparação de dados


Comando ```strip()``` elimina espaços no início e fim da string.

Comando ```dropna()``` remove registros com valores faltantes (*missing values*) no campo ```InvoiceNo```.

Comando ```df[~df['InvoiceNo'].str.contains('C')]``` remove registros com ```InvoiceNo``` iniciados com a letra *'C'*, uma vez que esses campos correspondem a pedidos cancelados.





In [0]:
df['Description'] = df['Description'].str.strip()
df.dropna(axis=0, subset=['InvoiceNo'], inplace=True)
df['InvoiceNo'] = df['InvoiceNo'].astype('str')
df = df[~df['InvoiceNo'].str.contains('C')]
print(df.describe())

            Quantity      UnitPrice     CustomerID
count  532621.000000  532621.000000  397924.000000
mean       10.239972       3.847621   15294.315171
std       159.593551      41.758023    1713.169877
min     -9600.000000  -11062.060000   12346.000000
25%         1.000000       1.250000   13969.000000
50%         3.000000       2.080000   15159.000000
75%        10.000000       4.130000   16795.000000
max     80995.000000   13541.330000   18287.000000


Gera uma base de dados apenas com pedidos da França. É gerada uma tabela pivô em que cada coluna corresponde à um produto e cada linha corresponde ao somatório da quantidade comprada daquele produto em um determinado pedido.


In [0]:
np.set_printoptions(threshold=None, precision=2)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)
pd.set_option('precision', 2)

basket = (df[df['Country'] =="France"]
          .groupby(['InvoiceNo', 'Description'])['Quantity']
          .sum().unstack().reset_index().fillna(0)
          .set_index('InvoiceNo'))
print(basket.head())

Transforma as quantidades em 0 ou 1.

In [0]:
def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1

basket_sets = basket.applymap(encode_units)
basket_sets.drop('POSTAGE', inplace=True, axis=1)

print(basket_sets.head())

Description  10 COLOUR SPACEBOY PEN  ...  ZINC T-LIGHT HOLDER STARS SMALL
InvoiceNo                            ...                                 
536370                            0  ...                                0
536852                            0  ...                                0
536974                            0  ...                                0
537065                            0  ...                                0
537463                            0  ...                                0

[5 rows x 1562 columns]


### Geração de ```itemsets``` frequentes e de regras de associação.



In [0]:
frequent_itemsets = apriori(basket_sets, min_support=0.07, use_colnames=True)
print(frequent_itemsets)

rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)
print("\nAlgumas regras de associação geradas:\n", rules.head())
print("\nDimensões da matriz de regras:", rules.shape)

     support                                           itemsets
0   0.071429                      (4 TRADITIONAL SPINNING TOPS)
1   0.096939                       (ALARM CLOCK BAKELIKE GREEN)
2   0.102041                        (ALARM CLOCK BAKELIKE PINK)
3   0.094388                         (ALARM CLOCK BAKELIKE RED)
4   0.081633                     (BAKING SET 9 PIECE RETROSPOT)
5   0.071429                     (CHILDRENS CUTLERY DOLLY GIRL)
6   0.099490                             (DOLLY GIRL LUNCH BOX)
7   0.096939                          (JUMBO BAG RED RETROSPOT)
8   0.076531                       (JUMBO BAG WOODLAND ANIMALS)
9   0.125000                           (LUNCH BAG APPLE DESIGN)
10  0.084184                      (LUNCH BAG DOLLY GIRL DESIGN)
11  0.153061                          (LUNCH BAG RED RETROSPOT)
12  0.119898                        (LUNCH BAG SPACEBOY DESIGN)
13  0.117347                               (LUNCH BAG WOODLAND)
14  0.142857                 (LUNCH BOX 

### Exemplos de filtros sobre regras de associação

In [0]:
rules[ (rules['lift'] >= 6) &
       (rules['confidence'] >= 0.8) ]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
2,(ALARM CLOCK BAKELIKE RED),(ALARM CLOCK BAKELIKE GREEN),0.094388,0.096939,0.079082,0.837838,8.642959,0.069932,5.568878
3,(ALARM CLOCK BAKELIKE GREEN),(ALARM CLOCK BAKELIKE RED),0.096939,0.094388,0.079082,0.815789,8.642959,0.069932,4.916181
16,(SET/6 RED SPOTTY PAPER PLATES),(SET/20 RED RETROSPOT PAPER NAPKINS),0.127551,0.132653,0.102041,0.800000,6.030769,0.085121,4.336735
18,(SET/6 RED SPOTTY PAPER PLATES),(SET/6 RED SPOTTY PAPER CUPS),0.127551,0.137755,0.122449,0.960000,6.968889,0.104878,21.556122
19,(SET/6 RED SPOTTY PAPER CUPS),(SET/6 RED SPOTTY PAPER PLATES),0.137755,0.127551,0.122449,0.888889,6.968889,0.104878,7.852041
20,"(SET/6 RED SPOTTY PAPER PLATES, SET/6 RED SPOT...",(SET/20 RED RETROSPOT PAPER NAPKINS),0.122449,0.132653,0.099490,0.812500,6.125000,0.083247,4.625850
21,"(SET/6 RED SPOTTY PAPER PLATES, SET/20 RED RET...",(SET/6 RED SPOTTY PAPER CUPS),0.102041,0.137755,0.099490,0.975000,7.077778,0.085433,34.489796
22,"(SET/6 RED SPOTTY PAPER CUPS, SET/20 RED RETRO...",(SET/6 RED SPOTTY PAPER PLATES),0.102041,0.127551,0.099490,0.975000,7.644000,0.086474,34.897959


In [0]:
basket['ALARM CLOCK BAKELIKE GREEN'].sum()

340.0

In [0]:
basket['ALARM CLOCK BAKELIKE RED'].sum()

316.0

### Análise de cesto de compras da Alemanha

Esse código é semelhante ao código utilizado para gerar as regras da França. O objetivo é mostrar como que o suporte mínimo e a confiança mínima podem variar de uma base para outra. Um país pode ter um perfil de compras mais homogêneo e gerar regras com suporte maior, enquanto outro país pode gerar regras com suporte menor.

In [0]:
basket2 = (df[df['Country'] =="Germany"]
          .groupby(['InvoiceNo', 'Description'])['Quantity']
          .sum().unstack().reset_index().fillna(0)
          .set_index('InvoiceNo'))

basket_sets2 = basket2.applymap(encode_units)
basket_sets2.drop('POSTAGE', inplace=True, axis=1)
frequent_itemsets2 = apriori(basket_sets2, min_support=0.05, use_colnames=True)
rules2 = association_rules(frequent_itemsets2, metric="lift", min_threshold=1)

rules2[ (rules2['lift'] >= 4) &
        (rules2['confidence'] >= 0.5)]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
1,(PLASTERS IN TIN CIRCUS PARADE),(PLASTERS IN TIN WOODLAND ANIMALS),0.115974,0.137856,0.067834,0.584906,4.242887,0.051846,2.076984
6,(PLASTERS IN TIN SPACEBOY),(PLASTERS IN TIN WOODLAND ANIMALS),0.107221,0.137856,0.061269,0.571429,4.145125,0.046488,2.011670
11,(RED RETROSPOT CHARLOTTE BAG),(WOODLAND CHARLOTTE BAG),0.070022,0.126915,0.059081,0.843750,6.648168,0.050194,5.587746
